In [1]:
from utils import utils
from utils import layer2
from utils.xgb_fn import*

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline

from jupyterthemes import jtplot
jtplot.reset()

from sklearn.model_selection import StratifiedKFold

In [41]:
train, test = utils.load_data()
train = train.drop(['id', 'target'], axis=1)
test = test.drop('id', axis=1)

Successfully loaded train and test data


In [42]:
train = train.drop(train.columns[train.columns.str.startswith('ps_calc')], axis=1)
train = train.drop(['ps_ind_10_bin','ps_ind_11_bin','ps_ind_12_bin','ps_ind_13_bin',
                        'ps_ind_18_bin','ps_car_03_cat','ps_car_10_cat','ps_ind_14'], axis=1)

test = test.drop(test.columns[test.columns.str.startswith('ps_calc')], axis=1)
test = test.drop(['ps_ind_10_bin','ps_ind_11_bin','ps_ind_12_bin','ps_ind_13_bin',
                        'ps_ind_18_bin','ps_car_03_cat','ps_car_10_cat','ps_ind_14'], axis=1)

In [43]:
interact_depth1 = ['ps_car_13|ps_reg_03','ps_car_14|ps_reg_03','ps_car_13|ps_car_14',
                   'ps_car_13|ps_ind_03','ps_ind_03|ps_reg_03','ps_ind_15|ps_reg_03',
                   'ps_car_13|ps_ind_05_cat','ps_ind_05_cat|ps_reg_03','ps_car_13|ps_car_13',
                   'ps_car_01_cat|ps_reg_03','ps_ind_03|ps_ind_15','ps_car_13|ps_ind_15',
                   'ps_reg_01|ps_reg_03','ps_car_14|ps_ind_03','ps_car_13|ps_ind_17_bin',
                   'ps_car_13|ps_reg_01','ps_ind_01|ps_ind_03','ps_car_01_cat|ps_car_13',
                   'ps_car_13|ps_reg_02','ps_car_15|ps_reg_03','ps_ind_03|ps_ind_03',]

interact_depth2 = ['ps_car_13|ps_ind_05_cat|ps_reg_03','ps_car_13|ps_car_14|ps_reg_03',
                   'ps_car_13|ps_ind_05_cat|ps_ind_17_bin','ps_car_13|ps_car_13|ps_reg_03',
                   'ps_car_13|ps_ind_03|ps_reg_03','ps_car_13|ps_ind_17_bin|ps_reg_03',
                   'ps_ind_05_cat|ps_ind_17_bin|ps_reg_03',]

interact_depth3 = ['ps_car_13|ps_ind_05_cat|ps_ind_17_bin|ps_reg_03',
                   'ps_car_13|ps_car_13|ps_ind_05_cat|ps_ind_17_bin',
                   'ps_car_13|ps_car_14|ps_ind_05_cat|ps_reg_03',
                   'ps_car_13|ps_car_13|ps_ind_05_cat|ps_reg_03',]

interact_depth4 = ['ps_car_13|ps_car_13|ps_ind_05_cat|ps_ind_17_bin|ps_reg_01',
                   'ps_car_13|ps_car_13|ps_car_14|ps_ind_05_cat|ps_reg_03',
                   'ps_car_13|ps_car_13|ps_ind_05_cat|ps_ind_15|ps_ind_17_bin',
                   'ps_car_13|ps_car_13|ps_ind_03|ps_ind_17_bin|ps_reg_02',
                   'ps_car_13|ps_car_13|ps_ind_05_cat|ps_ind_15|ps_reg_03',]

In [44]:
for interact in interact_depth1:
    train[interact] = train[interact.split('|')[0]]*train[interact.split('|')[1]]

In [45]:
for interact in interact_depth2:
    train[interact] = train[interact.split('|')[0]]*train[interact.split('|')[1]]*train[interact.split('|')[2]]

In [46]:
for interact in interact_depth3:
    train[interact] = train[interact.split('|')[0]]*train[interact.split('|')[1]]*train[interact.split('|')[2]]*train[interact.split('|')[3]]

In [47]:
for interact in interact_depth4:
    train[interact] = train[interact.split('|')[0]]*train[interact.split('|')[1]]*train[interact.split('|')[2]]*train[interact.split('|')[3]]*train[interact.split('|')[4]]

In [17]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=123)

In [73]:
param = {'tree_method': 'gpu_hist',
        'silent': True,
        'eval_metric': 'auc',
        'objective': 'binary:logistic',
        'eta': 0.01,
        'max_depth': 5,
        'gamma': 0.3,
        'min_child_weight': 7,
        'subsample': 0.5,
        'colsample_bytree': 0.5,
        'max_delta_step': 5
        }

In [21]:
score = {}
y = pd.read_csv('data/train.csv', na_values=-1).target

for col in train.columns:
    score_fold = 0
    folds = 1
    X = pd.concat([ps_reg_03,train[col]], axis=1)
    print('Variable: {}'.format(str(col)))
    for train_index, val_index in skf.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y[train_index], y[val_index]
        # Set Var for XGB
        xg_train = xgb.DMatrix(X_train, label=y_train)
        xg_val = xgb.DMatrix(X_val, label=y_val)
        watchlist = [(xg_train, 'train'), (xg_val, 'test')]
        #
        xgb_model = xgb.train(param, xg_train, 10000, watchlist, early_stopping_rounds=100
                            , feval=gini_xgb, maximize=True, verbose_eval=False)
        print('Fold{}: {}'.format(folds, xgb_model.best_score))
        score_fold += xgb_model.best_score/3
        folds += 1
    score[col] = score_fold
    print('Average Score: {}'.format(score[col]))

Variable: ps_ind_01
Fold1: 0.124542
Fold2: 0.123996
Fold3: 0.119359
Average Score: 0.12263233333333334
Variable: ps_ind_02_cat
Fold1: 0.11814
Fold2: 0.119609
Fold3: 0.11466
Average Score: 0.11746966666666667
Variable: ps_ind_03
Fold1: 0.137501
Fold2: 0.142853
Fold3: 0.137494
Average Score: 0.13928266666666667
Variable: ps_ind_04_cat
Fold1: 0.117846
Fold2: 0.122428
Fold3: 0.115082
Average Score: 0.118452
Variable: ps_ind_05_cat
Fold1: 0.146677
Fold2: 0.150795
Fold3: 0.1425
Average Score: 0.14665733333333333
Variable: ps_ind_06_bin
Fold1: 0.147362
Fold2: 0.146207
Fold3: 0.134452
Average Score: 0.14267366666666667
Variable: ps_ind_07_bin
Fold1: 0.151106
Fold2: 0.144481
Fold3: 0.140654
Average Score: 0.14541366666666666
Variable: ps_ind_08_bin
Fold1: 0.121984
Fold2: 0.126373
Fold3: 0.115504
Average Score: 0.121287
Variable: ps_ind_09_bin
Fold1: 0.121003
Fold2: 0.12399
Fold3: 0.117575
Average Score: 0.120856
Variable: ps_ind_15
Fold1: 0.13803
Fold2: 0.1403
Fold3: 0.127911
Average Score: 0.1

In [26]:
interact_score = pd.DataFrame.from_dict(score, orient='index')

In [33]:
interact_score = interact_score.sort_values(0, ascending=False)

In [55]:
interact_score[interact_score[0]>0.13].index

Index(['ps_car_13', 'ps_car_13|ps_car_13', 'ps_car_13|ps_ind_03',
       'ps_car_01_cat|ps_car_13', 'ps_car_13|ps_car_14', 'ps_car_13|ps_reg_01',
       'ps_car_12', 'ps_car_13|ps_car_13|ps_reg_03', 'ps_car_13|ps_ind_17_bin',
       'ps_car_14', 'ps_car_13|ps_ind_05_cat', 'ps_car_04_cat',
       'ps_ind_17_bin', 'ps_ind_05_cat', 'ps_ind_07_bin',
       'ps_car_13|ps_reg_03', 'ps_car_13|ps_reg_02', 'ps_car_14|ps_ind_03',
       'ps_car_15', 'ps_ind_06_bin', 'ps_car_01_cat', 'ps_ind_16_bin',
       'ps_car_06_cat', 'ps_car_11_cat', 'ps_car_07_cat',
       'ps_car_13|ps_ind_17_bin|ps_reg_03',
       'ps_car_13|ps_car_13|ps_ind_03|ps_ind_17_bin|ps_reg_02',
       'ps_ind_03|ps_ind_03', 'ps_ind_03', 'ps_car_13|ps_ind_03|ps_reg_03',
       'ps_car_02_cat', 'ps_ind_15', 'ps_car_13|ps_ind_05_cat|ps_ind_17_bin',
       'ps_car_13|ps_car_13|ps_ind_05_cat|ps_ind_17_bin', 'ps_car_09_cat',
       'ps_car_13|ps_car_13|ps_ind_05_cat|ps_ind_17_bin|ps_reg_01',
       'ps_car_13|ps_car_14|ps_reg_03',
  

In [64]:
dummies = pd.get_dummies(train, columns=['ps_car_04_cat', 'ps_ind_05_cat',
       'ps_car_01_cat', 'ps_car_06_cat', 'ps_car_11_cat', 'ps_car_07_cat',
       'ps_car_02_cat', 'ps_car_09_cat', 'ps_car_05_cat', 'ps_car_08_cat',
       'ps_ind_04_cat', 'ps_ind_02_cat'])

interact = ['ps_car_13', 'ps_car_13|ps_ind_03','ps_car_01_cat|ps_car_13', 'ps_car_13|ps_car_14', 
            'ps_car_13|ps_reg_01','ps_car_12', 'ps_car_13|ps_car_13|ps_reg_03', 
            'ps_car_13|ps_ind_17_bin','ps_car_14', 'ps_car_13|ps_ind_05_cat', 'ps_car_04_cat',
            'ps_ind_17_bin', 'ps_ind_05_cat', 'ps_ind_07_bin',
            'ps_car_13|ps_reg_03', 'ps_car_13|ps_reg_02', 'ps_car_14|ps_ind_03',
            'ps_car_15', 'ps_ind_06_bin', 'ps_car_01_cat', 'ps_ind_16_bin',
            'ps_car_06_cat', 'ps_car_11_cat', 'ps_car_07_cat',
            'ps_car_13|ps_ind_17_bin|ps_reg_03',
            'ps_car_13|ps_car_13|ps_ind_03|ps_ind_17_bin|ps_reg_02', 'ps_ind_03',]

In [71]:
X = pd.concat([train[interact],dummies],axis=1)
y = pd.read_csv('data/train.csv', na_values=-1).target

In [74]:
xgb_skf_train(X,y,param)

Training xgb Fold:1/5
[0]	train-auc:0.603292	test-auc:0.601205	train-gini:0.205978	test-gini:0.203991
Multiple eval metrics have been passed: 'test-gini' will be used for early stopping.

Will train until test-gini hasn't improved in 100 rounds.
[100]	train-auc:0.632408	test-auc:0.625161	train-gini:0.264817	test-gini:0.250321
[200]	train-auc:0.636325	test-auc:0.626786	train-gini:0.272649	test-gini:0.253573
[300]	train-auc:0.641842	test-auc:0.629521	train-gini:0.283685	test-gini:0.259043
[400]	train-auc:0.64929	test-auc:0.633065	train-gini:0.298581	test-gini:0.266129
[500]	train-auc:0.65606	test-auc:0.635875	train-gini:0.312119	test-gini:0.27175
[600]	train-auc:0.661696	test-auc:0.637589	train-gini:0.323392	test-gini:0.275177
[700]	train-auc:0.666734	test-auc:0.638738	train-gini:0.333468	test-gini:0.277476
[800]	train-auc:0.671035	test-auc:0.639686	train-gini:0.34207	test-gini:0.279371
[900]	train-auc:0.674648	test-auc:0.640091	train-gini:0.349297	test-gini:0.280183
[1000]	train-auc:0.6

KeyboardInterrupt: 